In [1]:
import tkinter as tk
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from quickdraw import QuickDrawDataGroup
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
from matplotlib import cm
from matplotlib.colors import to_hex

CATEGORIES = ["cat", "dog", "tree", "house", "car", "apple", "chair", "bird", "fish", "flower"]
NUM_CLASSES = len(CATEGORIES)
IMG_SIZE = (28, 28)
IMAGES_PER_CATEGORY = 1000

def load_and_preprocess_data():
    x_data, y_data = [], []
    for idx, category in enumerate(CATEGORIES):
        print(f"Loading {category} drawings")
        qd_group = QuickDrawDataGroup(category, max_drawings=IMAGES_PER_CATEGORY, recognized=True)
        images = [255 - np.array(d.get_image(stroke_width=3).resize(IMG_SIZE).convert("L")) for d in qd_group.drawings]
        x_data.extend(images)
        y_data.extend([idx] * len(images))
    x = np.array(x_data).reshape(-1, 28, 28, 1).astype('float32') / 255
    y = tf.keras.utils.to_categorical(y_data, NUM_CLASSES)
    print("load complete")
    return train_test_split(x, y, test_size=0.2, random_state=42)

def build_and_train_model():
    x_train, x_test, y_train, y_test = load_and_preprocess_data()
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', name='feature_layer')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

    feature_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('feature_layer').output)
    x_features = feature_model.predict(x_test)
    pca = PCA(n_components=2)
    x_tsne = pca.fit_transform(x_features)
    return model, feature_model, x_test, y_test, pca, x_tsne

class ObjectRecognizerApp:
    def __init__(self, root, model, feature_model, x_test, y_test, pca, x_tsne):
        self.model = model
        self.feature_model = feature_model
        self.x_test = x_test
        self.y_test = y_test
        self.pca = pca
        self.x_tsne = x_tsne
        self.root = root

        self.canvas = tk.Canvas(root, width=200, height=200, bg='white')
        self.canvas.pack()
        tk.Button(root, text="Predict", command=self.predict_object).pack()
        tk.Button(root, text="Clear", command=self.clear_canvas).pack()
        tk.Button(root, text="Open Doodle Dialog", command=self.open_doodle_dialog).pack()
        self.result_label = tk.Label(root, text="Draw and Predict")
        self.result_label.pack()

        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.last_x, self.last_y = None, None
        self.canvas.bind("<B1-Motion>", self.draw_line)
        self.canvas.bind("<ButtonRelease-1>", self.reset_last)

    def draw_line(self, event):
        if self.last_x is not None and self.last_y is not None:
            self.canvas.create_line(self.last_x, self.last_y, event.x, event.y, width=10, fill='black')
            self.draw.line([self.last_x, self.last_y, event.x, event.y], fill=0, width=10)
        self.last_x, self.last_y = event.x, event.y

    def reset_last(self, event):
        self.last_x, self.last_y = None, None

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="Draw and Predict")

    def preprocess_image(self, image):
        img = image.resize((28, 28))
        img_array = 255 - np.array(img)
        img_array = img_array.astype('float32') / 255
        return img_array.reshape(1, 28, 28, 1)

    def predict_object(self):
        img_array = self.preprocess_image(self.image)
        prediction = self.model.predict(img_array)
        idx = np.argmax(prediction)
        self.result_label.config(text=f"Predicted: {CATEGORIES[idx]} ({np.max(prediction):.2%})")

    def open_doodle_dialog(self):
        dialog = tk.Toplevel(self.root)
        dialog.geometry("900x600")
        canvas = tk.Canvas(dialog, width=200, height=200, bg='white')
        canvas.pack(side=tk.LEFT, padx=10)

        image = Image.new("L", (200, 200), 255)
        draw = ImageDraw.Draw(image)
        last = {"x": None, "y": None}

        def draw_line(event):
            if last["x"] is not None and last["y"] is not None:
                canvas.create_line(last["x"], last["y"], event.x, event.y, width=10, fill='black')
                draw.line([last["x"], last["y"], event.x, event.y], fill=0, width=10)
            last["x"], last["y"] = event.x, event.y

        def reset_last(event):
            last["x"], last["y"] = None, None

        canvas.bind("<B1-Motion>", draw_line)
        canvas.bind("<ButtonRelease-1>", reset_last)
        label = tk.Label(dialog, text="Draw and Predict")
        label.pack()

        def predict_and_visualize():
            arr = self.preprocess_image(image)
            pred = self.model.predict(arr)
            idx = np.argmax(pred)
            label.config(text=f"Predicted: {CATEGORIES[idx]} ({np.max(pred):.2%})")
            features = self.feature_model.predict(arr)
            self.display_plot_in_dialog(dialog, features, CATEGORIES[idx])

        tk.Button(dialog, text="Predict and Visualize", command=predict_and_visualize).pack()
        tk.Button(dialog, text="Clear", command=lambda: [canvas.delete("all"), image.paste(255, (0,0,200,200))]).pack()

    def display_plot_in_dialog(self, parent, doodle_features, doodle_label):
        all_features = np.vstack([self.feature_model.predict(self.x_test), doodle_features])
        all_tsne = self.pca.fit_transform(all_features)
        fig, ax = plt.subplots(figsize=(8, 6))
        cmap = cm.get_cmap('tab10', len(CATEGORIES))
        colors = [to_hex(cmap(i)) for i in range(len(CATEGORIES))]

        for i, cat in enumerate(CATEGORIES):
            mask = np.argmax(self.y_test, axis=1) == i
            pts = all_tsne[mask]
            ax.scatter(pts[:, 0], pts[:, 1], label=cat, s=30, alpha=0.6, edgecolors='w', linewidths=0.5, color=colors[i])
            if len(pts) >= 3:
                hull = ConvexHull(pts)
                polygon = Polygon(pts[hull.vertices], closed=True, edgecolor=colors[i], facecolor=colors[i], alpha=0.08)
                ax.add_patch(polygon)

        ax.scatter(all_tsne[-1, 0], all_tsne[-1, 1], marker='*', s=300, c='black', edgecolors='yellow', linewidths=2, zorder=10, label=f"Doodle: {doodle_label}")
        ax.set_title("Your Doodle in Feature Space", fontsize=14)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        ax.grid(True, linestyle='--', alpha=0.3)
        fig.tight_layout()

        canvas_frame = tk.Frame(parent)
        canvas_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        canvas = FigureCanvasTkAgg(fig, master=canvas_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

model, feature_model, x_test, y_test, pca, x_tsne = build_and_train_model()
root = tk.Tk()
app = ObjectRecognizerApp(root, model, feature_model, x_test, y_test, pca, x_tsne)
root.mainloop()


Loading cat drawings
loading cat drawings
load complete
Loading dog drawings
loading dog drawings
load complete
Loading tree drawings
loading tree drawings
load complete
Loading house drawings
loading house drawings
load complete
Loading car drawings
loading car drawings
load complete
Loading apple drawings
loading apple drawings
load complete
Loading chair drawings
loading chair drawings
load complete
Loading bird drawings
loading bird drawings
load complete
Loading fish drawings
loading fish drawings
load complete
Loading flower drawings
loading flower drawings
load complete
load complete
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2672 - loss: 2.0559 - val_accuracy: 0.5755 - val_loss: 1.3269
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5686 - loss: 1.2965 - val_accuracy: 0.6915 - val_loss: 0.9472
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6697 - loss: 0.9840 - val_accuracy: 0.7325 - val_loss: 0.7720
Epoch 4/10
63/63 ━━━━